# TEXT EMOTION DETECTION

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

### Loading the dataset

In [6]:
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TEXT-EMOTION/emotion_dataset_raw.csv")
df = pd.read_csv("C:/Users/samga/OneDrive/Desktop/capstone/Text-Emotion-Detection/Text Emotion Detection/data/emotion_dataset_raw.csv")

In [7]:
df.head()

,Emotion,Text
0,neutral,Why ?
1,joy,Sage Act upgrade on my to do list for tommorow.
2,sadness,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...
3,joy,Such an eye ! The true hazel eye-and so brill...
4,joy,@Iluvmiasantos ugh babe.. hugggzzz for u .! b...


In [8]:
df['Emotion'].value_counts()

joy         11045
sadness      6722
fear         5410
anger        4297
surprise     4062
neutral      2254
disgust       856
shame         146
Name: Emotion, dtype: int64

In [9]:
# sns.countplot(x='Emotion',data=df)

### Data pre-processing

In [10]:
!pip3 install neattext
import neattext.functions as nfx

# Remove the user handles
df['Clean_Text'] = df['Text'].apply(nfx.remove_userhandles)

     ---------------------------------------- 0.0/114.7 kB ? eta -:--:--
     ------------------------------- ------- 92.2/114.7 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 114.7/114.7 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'neattext'

In [ ]:
dir(nfx)

In [ ]:
# Remove the stopwords
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

In [ ]:
df

### Splitting data into input variables and target variable

x: Features are the attributes and variables extracted from the dataset. These extracted features are used as inputs to the model during training.

y: Labels are the output or the target variable.

In [ ]:
x = df['Clean_Text']
y = df['Emotion']

### Splitting data into train and test set

We need to split our dataset into a train set and test set. The model will learn from the train set. We will use the test set to evaluate the model performance and measure the model’s knowledge capability.

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
x_numpy = x.values

# Reshape the input data to 2D array if it's 1D
x_reshaped = x_numpy.reshape(-1, 1)

# Instantiate RandomOverSampler to balance the data
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(x_reshaped, y)

# Convert the resampled data back to pandas Series if needed
x_resampled_series = pd.Series(X_resampled.squeeze())

In [ ]:
sns.countplot(x='Emotion',data=df)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(x_resampled_series, y_resampled, test_size=0.2, random_state=42)

NameError: name 'x_resampled_series' is not defined

In [15]:
pipe_lr = Pipeline(steps=[
    ('cv', CountVectorizer()),
    ('lr', LogisticRegression())
])

### Training the model

In [16]:
# !pip install sklearn
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

In [17]:
# pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])
# pipe_lr.fit(X_train_resampled,y_train_resampled)
# pipe_lr.score(x_test,y_test)

In [18]:
pipe_lr.fit(X_train, y_train)
accuracy = pipe_lr.score(X_test, y_test)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8613626075147125


In [19]:
emotion_attached_training_data = []
# pipe_lr.fit(X_train, y_train)

for sentence in X_train:
  # Clean the sentence
  sentence = nfx.remove_userhandles(sentence)
  sentence = nfx.remove_stopwords(sentence)

  # Predict the emotion of the sentence
  emotion = pipe_lr.predict([sentence])[0]

  # Add the emotion to the sentence
  emotion_attached_sentence = f"{sentence} <{emotion}>"
  emotion_attached_training_data.append(emotion_attached_sentence)


df1 = pd.DataFrame(emotion_attached_training_data)

# Save the DataFrame to a CSV file
df1.to_csv('emotion_attached_training_data.csv', index=False)

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd

In [21]:
pipe_rf = Pipeline(steps=[('cv',CountVectorizer()),('rf', RandomForestClassifier(n_estimators=100, random_state=42))])
pipe_rf.fit(X_train, y_train)
predictions = pipe_rf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.893843368039837


In [22]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd

In [23]:
pipe_svm = Pipeline(steps=[
    ('cv', CountVectorizer()),
    ('svm', SVC(kernel='linear', random_state=42))  # SVM with linear kernel
])
pipe_svm.fit(X_train, y_train)
predictions = pipe_svm.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8793005885015844


In [24]:
# emotion_attached_training_data = []
# # pipe_lr.fit(X_train, y_train)

# for sentence in X_train:
#   # Clean the sentence
#   sentence = nfx.remove_userhandles(sentence)
#   sentence = nfx.remove_stopwords(sentence)

#   # Predict the emotion of the sentence
#   emotion = pipe_lr.predict([sentence])[0]

#   # Add the emotion to the sentence
#   emotion_attached_sentence = f"{sentence} <{emotion}>"
#   emotion_attached_training_data.append(emotion_attached_sentence)


# df1 = pd.DataFrame(emotion_attached_training_data)

# # Save the DataFrame to a CSV file
# df1.to_csv('emotion_attached_training_data.csv', index=False)

### Saving the model

In [25]:
import joblib
pipeline_file = open("text_emotion_balanced_dataset.pkl","wb")
joblib.dump(pipe_lr,pipeline_file)
pipeline_file.close()

In [26]:
# def attach_emotions(text):
#   # Clean the text
#   text = nfx.remove_userhandles(text)
#   text = nfx.remove_stopwords(text)

#   # Predict the emotion of the text
#   emotion = pipe_lr.predict([text])[0]

#   # Add the emotion to the text
#   text = f"{text} ({emotion})"

#   return text

# # Example usage
# text = "I am so happy today!"
# emotion_attached_text = attach_emotions(text)
# print(emotion_attached_text)


In [27]:
# from google.colab import drive
# drive.mount('/content/drive')